# Scraping collegedata.com with Qt Webkit


In [1]:
%matplotlib inline
import numpy as np
import scipy as sp
import matplotlib as mpl
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import pandas as pd
pd.set_option('display.width', 500)
pd.set_option('display.max_columns', 100)
pd.set_option('display.notebook_repr_html', True)
import seaborn as sns
sns.set_style("whitegrid")
sns.set_context("poster")

import requests
import time
from bs4 import BeautifulSoup
import re

In [2]:
college_id_dict={'Princeton': 111, 'Harvard': 444, 'Yale': 244, 'Columbia': 399, 'Stanford': 781, 'UChicago': 327, 'MIT': 186, 'Duke': 1026, 'UPenn': 67, 'CalTech': 706, 'JohnsHopkins': 1509, 'Dartmouth': 403, 'Northwestern': 1803, 'Brown': 163, 'Cornell': 787, 'Vanderbilt': 1562, 'WashU': 1720, 'Rice': 731, 'NotreDame': 1774, 'UCB': 1090, 'Emory': 1039, 'Georgetown': 1182, 'CarnegieMellon': 204, 'UCLA': 1093, 'USC': 1138}
baseurl='http://www.collegedata.com/cs/admissions/'
tracker_url='admissions_tracker_result.jhtml?schoolId='
student_url='admissions_profile_view.jhtml?profileName='

In [3]:
## Source: https://webscraping.com/blog/Scraping-multiple-JavaScript-webpages-with-webkit/
import sys
from PyQt4.QtCore import *
from PyQt4.QtGui import *
from PyQt4.QtWebKit import *

class Render(QWebPage):  
    def __init__(self, urls):
        self.app = QApplication(sys.argv)  
        QWebPage.__init__(self)  
        self.loadFinished.connect(self._loadFinished)  
        self.urls = urls 
        self.profileList=set()
        self.crawl()  
        self.app.exec_()  
      
    def crawl(self):  
        if self.urls:  
            url = self.urls.pop(0)    
            self.mainFrame().load(QUrl(url))  
        else:  
            self.app.quit()  
        
    def _loadFinished(self, result): 
        frame = self.mainFrame()  
        url = str(frame.url().toString())  
        html = frame.toHtml()  
        self.scrape(url, html)
        self.crawl()
    
    def scrape(self,url, html):
        soup = BeautifulSoup(str(html.toAscii()),'html.parser')     
        profiles=soup.find_all("a",href=re.compile(r"enterProfileByName"))
        for p in profiles:
            self.profileList.add(p.get("href").split("'")[1])

Execute the next cell to try out for 1 university and 2 years

In [ ]:
#urls=[]
#schoolid=111
#schoolurl=baseurl+tracker_url+str(schoolid)+'&classYear='
#for year in range(2009,2011):
#    urls.append(schoolurl+str(year))
#r=Render(urls)
#print len(r.profileList)

Execute the next cell to try out for all universities and years 2009-2019. This takes about 55 minutes.

In [4]:
%%time
urls=[]
for school in college_id_dict:
    schoolurl=baseurl+tracker_url+str(college_id_dict[school])+'&classYear='
    for year in range(2009,2020):
        urls.append(schoolurl+str(year))
r=Render(urls)
print 'We found ', len(r.profileList), 'students'

We found  5020 students
CPU times: user 21min 40s, sys: 3min 8s, total: 24min 48s
Wall time: 53min 21s
